##### **Task_1_Training**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_train.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/y_test.dat
!wget https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/metadata.csv

--2021-10-14 22:56:17--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_train.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.88.184
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.88.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8571600000 (8.0G) [application/x-www-form-urlencoded]
Saving to: ‘X_train.dat’

X_train.dat         100%[===================>]   7.98G  92.0MB/s    in 86s     

2021-10-14 22:57:43 (95.1 MB/s) - ‘X_train.dat’ saved [8571600000/8571600000]

--2021-10-14 22:57:44--  https://dsci6015aisecf21.s3.us-east-2.amazonaws.com/X_test.dat
Resolving dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)... 52.219.99.18
Connecting to dsci6015aisecf21.s3.us-east-2.amazonaws.com (dsci6015aisecf21.s3.us-east-2.amazonaws.com)|52.219.99.18|:443... connected.
HTTP request sent, awa

In [5]:
!cp /content/X_train.dat /content/drive/MyDrive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/MyDrive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/MyDrive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/MyDrive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/MyDrive/vMalConv/metadata.csv

In [6]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2021-10-14 23:00:26--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/elastic/ember/archive/master.zip [following]
--2021-10-14 23:00:26--  https://github.com/elastic/ember/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/elastic/ember/zip/master [following]
--2021-10-14 23:00:26--  https://codeload.github.com/elastic/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [     <=>            ]  11.22M  8.81MB/s    i

In [7]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/MyDrive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/MyDrive/vMalConv/")

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
labelrows = (y_train != -1)
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [9]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [10]:
!cp /content/X_train.h5 /content/drive/MyDrive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/MyDrive/vMalConv/y_train.h5

In [11]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  # Model architecture
  from tensorflow.keras import layers
  
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [12]:
model = make_model()
model.save('./content/drive/MyDrive/vMalConv/model.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [14]:
X_train = mms.transform(X_train)

In [15]:
## Reshape to create 3 channels ##
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

In [16]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

save_dir = "drive/MyDrive/vMalConv/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
# Save the weights #
model.save_weights (save_dir+'weights.h5')

# Save the model architecture #
model_json = model.to_json()
with open(save_dir+"model.json", "w") as json_file:
    json_file.write(model_json)

print("model saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
479744/480000 [============================>.] - ETA: 0s - loss: 7.1075 - accuracy: 0.9411 - auc_1: 0.9471 - precision_1: 0.9398

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


480000/480000 [==============================] - 62s 128us/sample - loss: 7.1049 - accuracy: 0.9411 - auc_1: 0.9471 - precision_1: 0.9398 - val_loss: 2.6292 - val_accuracy: 0.9738 - val_auc_1: 0.9760 - val_precision_1: 0.9686
Epoch 2/30
480000/480000 [==============================] - 64s 133us/sample - loss: 8.9062 - accuracy: 0.9579 - auc_1: 0.9618 - precision_1: 0.9568 - val_loss: 4.5756 - val_accuracy: 0.9850 - val_auc_1: 0.9865 - val_precision_1: 0.9872
Epoch 3/30
480000/480000 [==============================] - 72s 150us/sample - loss: 10.1289 - accuracy: 0.9630 - auc_1: 0.9666 - precision_1: 0.9622 - val_loss: 5.3183 - val_accuracy: 0.9828 - val_auc_1: 0.9870 - val_precision_1: 0.9861
Epoch 4/30
480000/480000 [==============================] - 69s 143us/sample - loss: 8.9704 - accuracy: 0.9663 - auc_1: 0.9708 - precision_1: 0.9651 - val_loss: 3.4918 - val_accuracy: 0.9835 - val_auc_1: 0.9888 - val_precision_1: 0.9886
Epoch 5/30
480000/480000 [==============================] - 67

In [14]:
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe

--2021-10-15 00:10:54--  https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe
Resolving the.earth.li (the.earth.li)... 93.93.131.124, 2a00:1098:86:4d:c0ff:ee:15:900d
Connecting to the.earth.li (the.earth.li)|93.93.131.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://the.earth.li/~sgtatham/putty/0.76/w32/putty.exe [following]
--2021-10-15 00:10:55--  https://the.earth.li/~sgtatham/putty/0.76/w32/putty.exe
Reusing existing connection to the.earth.li:443.
HTTP request sent, awaiting response... 200 OK
Length: 1180904 (1.1M) [application/x-msdos-program]
Saving to: ‘putty.exe.2’

putty.exe.2         100%[===================>]   1.13M  1.95MB/s    in 0.6s    

2021-10-15 00:10:55 (1.95 MB/s) - ‘putty.exe.2’ saved [1180904/1180904]



In [17]:
def test_pefile(pefile):
  try:
    import ember
    import numpy as np
  except:
    !wget https://github.com/endgameinc/ember/archive/master.zip
    !unzip master.zip
    !rm master.zip
    !cp -r ember-master/* .
    !rm -r ember-master
    !pip install -r requirements.txt
    !python setup.py install
    
    import ember

  from sklearn.preprocessing import StandardScaler
  import tensorflow as tf
  tf.compat.v1.disable_eager_execution()

  mms=StandardScaler()
  sample_data = open(pefile, "rb").read()
  extractor = ember.PEFeatureExtractor(2)
  sample_data = np.array(extractor.feature_vector(sample_data), dtype=np.float32)
  sample_data = mms.fit_transform([sample_data])
  sample_data = np.reshape(sample_data,(-1,1,2381))

  model = tf.keras.models.load_model('./content/drive/MyDrive/vMalConv/model.h5')
  pred = model.predict(sample_data)

  return pred

In [18]:

test_pefile("putty.exe")

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


array([[[0.5]]], dtype=float32)

Benign Sample
